In this notebook, Using the Gemini API's automatic function calling to build a chat interface over a local database. This shows how to add AI chat capabilities to existing applications with ease and how it will simplify things in real world.

# Importing Library 

In [1]:
# Remove unused conflicting packages
!pip uninstall -qqy jupyterlab  
!pip install -U -q "google-genai==1.7.0"
!pip install kaggle
# !pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.6 MB/s eta 0:00:00


In [2]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

# Setting up API key

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# Automated retry

In [4]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

# Create a local database

For this Chatbot, created a local SQLite database and add some synthetic data which consist of 3 table (products,staff,orders). Load the sql IPython extension so we can interact with the database using magic commands (the % instructions) to create a new, empty SQLite database.

In [5]:
%load_ext sql
%sql sqlite:///sample.db

In [6]:
%%sql
-- Create the 'products' table
CREATE TABLE IF NOT EXISTS products (
  	product_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	product_name VARCHAR(255) NOT NULL,
  	price DECIMAL(10, 2) NOT NULL,
    company_name VARCHAR(10),
    guarantee_years INTEGER,
    warranty_years INTEGER   
  );

-- Create the 'staff' table
CREATE TABLE IF NOT EXISTS staff (
  	staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	first_name VARCHAR(255) NOT NULL,
  	last_name VARCHAR(255) NOT NULL,
    years_experience INTEGER
  );

-- Create the 'orders' table
CREATE TABLE IF NOT EXISTS orders (
  	order_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	customer_name VARCHAR(255) NOT NULL,
  	staff_id INTEGER NOT NULL,
  	product_id INTEGER NOT NULL,
    order_number VARCHAR(20),
    total_price DECIMAL(10, 2),
  	FOREIGN KEY (staff_id) REFERENCES staff (staff_id),
  	FOREIGN KEY (product_id) REFERENCES products (product_id)
  );


-- Insert data into the 'products' table
INSERT INTO products (product_name, price, company_name, guarantee_years, warranty_years) 
VALUES     ('Ultra HD Smart TV', 1299.99, 'SL', 2, 1),
    ('Wireless Earbuds', 89.99, 'AP', 1, 1),
    ('Gaming Console', 499.99, 'MS', 3, 2),
    ('Smart Watch', 199.99, 'AP', 1, 1),
    ('Blender', 49.99, 'HM', 1, 0),
    ('Air Purifier', 179.99, 'DL', 2, 1),
    ('Robot Vacuum', 299.99, 'EC', 2, 2),
    ('Fitness Tracker', 79.99, 'FB', 1, 1),
    ('Coffee Maker', 129.99, 'KM', 1, 0),
    ('External SSD 1TB', 149.99, 'SM', 3, 2);

-- Insert data into the 'staff' table
INSERT INTO staff (first_name, last_name, years_experience) VALUES
    ('Diana', 'Miller', 5),
    ('Ethan', 'Davis', 3),
    ('Grace', 'Wilson', 7),
    ('Henry', 'Brown', 2),
    ('Isabella', 'Taylor', 4),
    ('Jack', 'Anderson', 6);

-- Insert data into the 'orders' table
INSERT INTO orders (customer_name, staff_id, product_id, order_number, total_price) VALUES
    ('Olivia Martinez', 1, 1, 'ORD1001', 1299.99),
    ('Liam Thompson', 2, 3, 'ORD1002', 499.99),
    ('Sophia Garcia', 3, 5, 'ORD1003', 49.99),
    ('Noah Robinson', 4, 2, 'ORD1004', 89.99),
    ('Emma Clark', 5, 4, 'ORD1005', 199.99),
    ('Mason Rodriguez', 6, 6, 'ORD1006', 179.99),
    ('Ava Lewis', 1, 7, 'ORD1007', 299.99),
    ('Jacob Walker', 2, 8, 'ORD1008', 79.99),
    ('Charlotte Hall', 3, 9, 'ORD1009', 129.99),
    ('William Young', 4, 10, 'ORD1010', 149.99),
    ('Amelia Allen', 5, 1, 'ORD1011', 1299.99),
    ('Benjamin King', 6, 3, 'ORD1012', 499.99),
    ('Mia Scott', 1, 5, 'ORD1013', 49.99),
    ('James Green', 2, 7, 'ORD1014', 299.99),
    ('Harper Adams', 3, 9, 'ORD1015', 129.99),
    ('Elijah Nelson', 4, 2, 'ORD1016', 89.99),
    ('Abigail Baker', 5, 4, 'ORD1017', 199.99),
    ('Lucas Carter', 6, 6, 'ORD1018', 179.99),
    ('Emily Mitchell', 1, 8, 'ORD1019', 79.99),
    ('Daniel Perez', 2, 10, 'ORD1020', 149.99),
    ('Elizabeth Roberts', 3, 1, 'ORD1021', 1299.99),
    ('Alexander Turner', 4, 3, 'ORD1022', 499.99),
    ('Evelyn Phillips', 5, 5, 'ORD1023', 49.99),
    ('Michael Campbell', 6, 7, 'ORD1024', 299.99),
    ('Sofia Parker', 1, 9, 'ORD1025', 129.99),
    ('Matthew Evans', 2, 2, 'ORD1026', 89.99),
    ('Avery Edwards', 3, 4, 'ORD1027', 199.99),
    ('David Collins', 4, 6, 'ORD1028', 179.99),
    ('Scarlett Stewart', 5, 8, 'ORD1029', 79.99),
    ('Joseph Sanchez', 6, 10, 'ORD1030', 149.99),
    ('Victoria Morris', 1, 1, 'ORD1031', 1299.99),
    ('Jackson Rogers', 2, 3, 'ORD1032', 499.99),
    ('Madison Reed', 3, 5, 'ORD1033', 49.99),
    ('Samuel Cook', 4, 7, 'ORD1034', 299.99),
    ('Luna Morgan', 5, 9, 'ORD1035', 129.99),
    ('Sebastian Bell', 6, 2, 'ORD1036', 89.99),
    ('Penelope Murphy', 1, 4, 'ORD1037', 199.99),
    ('Henry Bailey', 2, 6, 'ORD1038', 179.99),
    ('Layla Rivera', 3, 8, 'ORD1039', 79.99),
    ('Owen Cooper', 4, 10, 'ORD1040', 149.99),
    ('Chloe Richardson', 5, 1, 'ORD1041', 1299.99),
    ('Gabriel Cox', 6, 3, 'ORD1042', 499.99),
    ('Riley Howard', 1, 5, 'ORD1043', 49.99),
    ('Luke Ward', 2, 7, 'ORD1044', 299.99),
    ('Zoey Torres', 3, 9, 'ORD1045', 129.99),
    ('Julian Peterson', 4, 2, 'ORD1046', 89.99),
    ('Stella Gray', 5, 4, 'ORD1047', 199.99),
    ('Levi Ramirez', 6, 6, 'ORD1048', 179.99),
    ('Hazel James', 1, 8, 'ORD1049', 79.99),
    ('Wyatt Watson', 2, 10, 'ORD1050', 149.99);

 * sqlite:///sample.db
Done.
Done.
Done.
10 rows affected.
6 rows affected.
50 rows affected.


[]

# Define database functions

Below is the 3 python function which we use to answer the query from the user. This function run query on the sample database. Firstly openig connection.

In [7]:
import sqlite3

db_file = "sample.db"
db_conn = sqlite3.connect(db_file)

def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables()')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]


list_tables()
# listoftables = list_tables()

 - DB CALL: list_tables()


['products', 'sqlite_sequence', 'staff', 'orders']

In [8]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(f' - DB CALL: describe_table({table_name})')

    cursor = db_conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    # [column index, column name, column type, ...]
    return [(col[1], col[2]) for col in schema]

# describe_table()
# for i in listoftables:
#     print(f'Describing the table: {i}')
#     print(describe_table(str(i)))

In [9]:
def execute_query(sql: str) -> list[list[str]]:
    """Execute an SQL statement, returning the results."""
    print(f' - DB CALL: execute_query({sql})')

    cursor = db_conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()


execute_query("select * from products")

 - DB CALL: execute_query(select * from products)


[(1, 'Ultra HD Smart TV', 1299.99, 'SL', 2, 1),
 (2, 'Wireless Earbuds', 89.99, 'AP', 1, 1),
 (3, 'Gaming Console', 499.99, 'MS', 3, 2),
 (4, 'Smart Watch', 199.99, 'AP', 1, 1),
 (5, 'Blender', 49.99, 'HM', 1, 0),
 (6, 'Air Purifier', 179.99, 'DL', 2, 1),
 (7, 'Robot Vacuum', 299.99, 'EC', 2, 2),
 (8, 'Fitness Tracker', 79.99, 'FB', 1, 1),
 (9, 'Coffee Maker', 129.99, 'KM', 1, 0),
 (10, 'External SSD 1TB', 149.99, 'SM', 3, 2)]

# Implement function calls

Now putting it all together in a call to the Gemini API.

Function calling works by adding specific messages to a chat session. When function schemas are defined and made available to the model and a conversation is started, instead of returning a text response, the model may return a `function_call` instead. When this happens, the client must respond with a `function_response`, indicating the result of the call, and the conversation can continue on as normal.

This function calling interaction normally happens manually, allowing you, the client, to validate and initiate the call. However the Python SDK also supports **automatic function calling**, where the supplied functions will be automatically invoked. This is a powerful feature and should be used with care, such as when the functions have no [side-effects](https://en.wikipedia.org/wiki/Side_effect_(computer_science)).

Here's the state diagram representing the conversation flow with function calling. With automatic function calling, the bottom row is executed automatically by the Python SDK. With manual function calling, you write the code to run each step individually.

![function calling state diagram](https://codelabs.developers.google.com/static/codelabs/gemini-function-calling/img/gemini-function-calling-overview_1440.png)

This below is the prompt/instruction for the chatbot to follow while andressing any query raised by the user, the instruction also contain the table structure and column descriptions.

In [10]:

db_tools = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot that can interact with an SQL database
for a electronic store. You will take the users questions and turn them into SQL
queries using the tools available. Once you have the information you need, you will
answer the user's question using the data returned.

Use list_tables to see what tables are present, describe_table to understand the
schema, and execute_query to issue an SQL SELECT query.

Further find the below Table Descriptions to futher unserstand the query

### products table
- `product_id`: Unique identifier for each product (primary key).
- `product_name`: Name of the product (cannot be null).
- `price`: Cost of the product with 2 decimal places precision (cannot be null).
- `company_name`: Initials of the manufacturing company.
- `guarantee_years`: Number of years the product is guaranteed.
- `warranty_years`: Number of years the product is under warranty.

### staff table
- `staff_id`: Unique identifier for each staff member (primary key).
- `first_name`: Staff member's first name (cannot be null).
- `last_name`: Staff member's last name (cannot be null).
- `years_experience`: Number of years the staff member has worked in the field.

### orders table
- `order_id`: Unique identifier for each order (primary key).
- `customer_name`: Name of the customer placing the order (cannot be null).
- `staff_id`: Identifier linking to the staff member who handled the order (foreign key).
- `product_id`: Identifier linking to the product ordered (foreign key).
- `order_number`: Unique tracking number for the order.
- `total_price`: Final price of the order including any adjustments.
"""

# Defining Function to address query

In [11]:
def Chatbotq(Query):
    client = genai.Client(api_key=GOOGLE_API_KEY)
    # Start a chat with automatic function calling enabled.
    chat = client.chats.create(model="gemini-2.0-flash",
    config=types.GenerateContentConfig(system_instruction=instruction,
        tools=db_tools,),)
    resp = chat.send_message(Query)
    return resp.text

q = '0'
while 1>0:
    # To close the chat user might type this to close
    if q in {"q", "quit", "exit", "goodbye"}:
        break
    else:
        q = input()
        print(f"\n{Chatbotq(q)}")
        

 what is the maximum order price?


 - DB CALL: execute_query(SELECT max(total_price) FROM orders)

The maximum order price is 1299.99.


 q



OK. What would you like to know?


In [12]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 74.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.5 MB/s eta 0:00:00:00:0100:01


In [13]:
import streamlit as st

# Initializing streamlit app
st.set_page_config(page_title= "Q&A Demo")
st.header("Chatbot")

#Initializing session state for chat history if it does not exist
if 'chat_history' not in st.session_state:
    st.session_state['chat_history'] = []

input = st.text_input("Input:", key = 'input')
submit = st.button("Ask")

if submit and input:
    response = Chatbotq(input)
    #Adding our queries to chat history
    st.session_state['chat_history'].append(("You:", input))
    st.subheader("Response:")
    for chunk in response:
        st.write(chunk.text)
        st.session_state['chat_history'].append(("Bot:", chunk.text))
st.subheader("Conversation")

for role,text in st.session_state['chat_history']:
    st.write(f"{role}:{text}")

2025-05-11 10:04:14.426 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 10:04:14.428 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 10:04:14.635 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-05-11 10:04:14.636 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 10:04:14.638 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 10:04:14.638 Session state does not function when running a script without `streamlit run`
2025-05-11 10:04:14.639 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-0